In [21]:
%run ../talktools.py
!rm galaxy1000.csv
!rm test.csv
!rm -r joblib_cache

rm: galaxy1000.csv: No such file or directory
rm: test.csv: No such file or directory
rm: joblib_cache: No such file or directory


# A (Simplistic) Introduction to Data Caching

<div style="text-align: center"><font size=-1>AY128/256 UC Berkeley (2022)</font> </div> 

<quote>
    <i><font color="grey">"There are two hard things in Computer Science, cache invalidation, naming things, and off-by-one errors."</i></font>
    <div style="text-align: right"><font color="grey">--Socrates</font></div>

</quote>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ucb-datalab/course_materials_2022/blob/main/Lectures/Lecture0_Viz_and_Gaia/02_caching.ipynb)

<b>Motivation</b>: In data-intensive workloads, like the ones you'll be building in this class, it's a good idea to think about your data caching strategies up front. During exploratory analysis, especially when interacting with external sources of databases (e.g., flat files hosted remotely, external databases, etc.) sensible caching can save you a bunch of time in not needing to (re)download data. 

Even for pure local interactions, you might want to cache the results of computation to avoid having to rerun long-running/expensive calcuations.

In the context of reproducbility/replicability, having a cache/store of the data inputs will make it easier for others to redo the steps you did and get the same results.

## Persistent or Ephemeral?

Every time you run an expensive computation or do a time-intensive query, you implicitly store the results in emphemeral Python object.

Let's get a CSV file with some data about galaxies from SDSS:

In [22]:
import time
import os
import io

import pandas as pd
import requests  # requests is the standard URL

external_file_location = "https://raw.githubusercontent.com/AstroHackWeek/AstroHackWeek2014/master/day4/galaxy1000.csv"
local_filename = os.path.basename(external_file_location)

In [23]:
# ATTEMPT 1: download the CSV and load it into a DataFrame
r = requests.get(external_file_location)
df = pd.read_csv(io.StringIO(r.text))
df

,objid,ra,dec,dered_u,dered_g,dered_r,dered_i,dered_z,mag_u,mag_g,...,u_g_color,g_r_color,r_i_color,i_z_color,class,diff_u,diff_g,diff_g1,diff_i,diff_z
0,1237648720142401611,146.71421,-1.041304,16.86573,15.28762,14.52209,14.11909,13.82587,18.80072,17.20840,...,1.578108,0.765535,0.402999,0.293222,GALAXY,-1.934988,-1.920773,-1.917164,-1.800784,-1.778052
1,1237650795146510627,146.74413,-0.652191,20.90252,18.81306,17.48005,16.97422,16.63207,21.91268,19.99857,...,2.089464,1.333010,0.505831,0.342144,GALAXY,-1.010160,-1.185516,-1.221382,-1.223454,-1.225744
2,1237650795146445031,146.62857,-0.765137,19.12518,17.31025,16.46868,16.04988,15.72728,20.51632,18.86351,...,1.814936,0.841570,0.418800,0.322595,GALAXY,-1.391132,-1.553261,-1.482050,-1.514194,-1.487653
3,1237648720142401670,146.63167,-0.988278,19.21963,18.12533,17.69570,17.41701,17.34536,20.68625,19.73145,...,1.094292,0.429634,0.278688,0.071650,GALAXY,-1.466625,-1.606113,-1.745596,-1.619215,-1.610914
4,1237650795683512690,147.03779,-0.266591,19.60107,18.41345,17.73770,17.43207,17.32257,21.96092,20.84575,...,1.187624,0.675747,0.305634,0.109497,GALAXY,-2.359842,-2.432301,-2.344486,-2.367418,-2.207579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1237657775542632759,134.09469,39.946506,18.39273,16.39338,15.42325,14.98728,14.61381,20.33722,18.20288,...,1.999353,0.970126,0.435975,0.373470,GALAXY,-1.944487,-1.809502,-1.971534,-2.052320,-1.971382
996,1237657775542698090,134.27512,40.022711,20.69093,18.47890,17.51366,17.10299,16.73161,21.46971,19.17122,...,2.212025,0.965242,0.410664,0.371384,GALAXY,-0.778788,-0.692322,-0.944075,-0.895832,-0.830559
997,1237657775542698177,134.18695,39.944569,18.67964,17.48961,17.15747,16.90512,16.83414,21.07121,20.06086,...,1.190033,0.332136,0.252352,0.070980,GALAXY,-2.391565,-2.571253,-2.977261,-2.889906,-2.671612
998,1237657630586634463,132.98594,39.814486,19.05170,17.87203,17.19312,16.79870,16.52653,20.61515,19.74957,...,1.179663,0.678915,0.394419,0.272171,GALAXY,-1.563450,-1.877533,-1.913368,-1.791895,-1.615683


Great: we can use the pandas DataFrame `df` in the downstream analysis. The **problem of course is that we rerun our notebook, we need to redownload the data**. This ephemeral caching is usually just fine when the data is small and the computations are quick, but generally slows us down with larger files and beefier computations. 

Why don't we modify the above to check to see if the file exists and load it in if so. That is, let's create a persistent store of our data:

In [24]:
def get_galaxy_dataframe():
    """
    get the galaxy CSV file from SDSS if we dont have it already
    and parse it into a pandas DataFrame. Save the file
    for future use.
    
    Returns: dataframe
    """
    if not os.path.exists(local_filename):
        start = time.time()
        r = requests.get(external_file_location)

        # note: below is for smallish files. To download and save larger files
        # see https://stackoverflow.com/a/14114741
        with open(local_filename, 'w') as handle:
            handle.write(r.text)

        print(f"Wrote the file {local_filename} to disk")
        # from the Python object `r` instead of from disk to avoid disk IO
        df = pd.read_csv(io.StringIO(r.text))
        print(f"  Total time: {time.time() - start:0.3} sec")
    else:
        start = time.time()
        print(f"Reading the file {local_filename} from disk")
        df = pd.read_csv(local_filename)
        print(f"  Total time: {time.time() - start:0.3} sec")
    return df

In [25]:
#!rm galaxy1000.csv
get_galaxy_dataframe()

Wrote the file galaxy1000.csv to disk
  Total time: 0.0587 sec


,objid,ra,dec,dered_u,dered_g,dered_r,dered_i,dered_z,mag_u,mag_g,...,u_g_color,g_r_color,r_i_color,i_z_color,class,diff_u,diff_g,diff_g1,diff_i,diff_z
0,1237648720142401611,146.71421,-1.041304,16.86573,15.28762,14.52209,14.11909,13.82587,18.80072,17.20840,...,1.578108,0.765535,0.402999,0.293222,GALAXY,-1.934988,-1.920773,-1.917164,-1.800784,-1.778052
1,1237650795146510627,146.74413,-0.652191,20.90252,18.81306,17.48005,16.97422,16.63207,21.91268,19.99857,...,2.089464,1.333010,0.505831,0.342144,GALAXY,-1.010160,-1.185516,-1.221382,-1.223454,-1.225744
2,1237650795146445031,146.62857,-0.765137,19.12518,17.31025,16.46868,16.04988,15.72728,20.51632,18.86351,...,1.814936,0.841570,0.418800,0.322595,GALAXY,-1.391132,-1.553261,-1.482050,-1.514194,-1.487653
3,1237648720142401670,146.63167,-0.988278,19.21963,18.12533,17.69570,17.41701,17.34536,20.68625,19.73145,...,1.094292,0.429634,0.278688,0.071650,GALAXY,-1.466625,-1.606113,-1.745596,-1.619215,-1.610914
4,1237650795683512690,147.03779,-0.266591,19.60107,18.41345,17.73770,17.43207,17.32257,21.96092,20.84575,...,1.187624,0.675747,0.305634,0.109497,GALAXY,-2.359842,-2.432301,-2.344486,-2.367418,-2.207579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1237657775542632759,134.09469,39.946506,18.39273,16.39338,15.42325,14.98728,14.61381,20.33722,18.20288,...,1.999353,0.970126,0.435975,0.373470,GALAXY,-1.944487,-1.809502,-1.971534,-2.052320,-1.971382
996,1237657775542698090,134.27512,40.022711,20.69093,18.47890,17.51366,17.10299,16.73161,21.46971,19.17122,...,2.212025,0.965242,0.410664,0.371384,GALAXY,-0.778788,-0.692322,-0.944075,-0.895832,-0.830559
997,1237657775542698177,134.18695,39.944569,18.67964,17.48961,17.15747,16.90512,16.83414,21.07121,20.06086,...,1.190033,0.332136,0.252352,0.070980,GALAXY,-2.391565,-2.571253,-2.977261,-2.889906,-2.671612
998,1237657630586634463,132.98594,39.814486,19.05170,17.87203,17.19312,16.79870,16.52653,20.61515,19.74957,...,1.179663,0.678915,0.394419,0.272171,GALAXY,-1.563450,-1.877533,-1.913368,-1.791895,-1.615683


Hey, the capability (download and save, otherwise load) might be something we want to use a lot. Let's make the function a big more generic:

In [26]:
def get_and_parse_cached_remote_csvfile(url, local_filename=None, verbose=True):
    """
    get a CSV file at `url` if we dont have it already
    and parse it into a pandas DataFrame. Save the file
    for future use. Guess the output filename if not given.
    
    Returns: dataframe
    """
    
    # here we might error check to see if the URL is valid, points to a CSV file
    # etc.
    if local_filename is None:
        local_filename = os.path.basename(url)
        
    if not os.path.exists(local_filename):
        start = time.time()
        r = requests.get(url)

        # note: below is for smallish files. To download and save larger files
        # see https://stackoverflow.com/a/14114741
        with open(local_filename, 'w') as handle:
            handle.write(r.text)
        if verbose:
            print(f"Wrote the file {local_filename} to disk", flush=True)
        # from the Python object `r` instead of from disk to avoid disk IO
        df = pd.read_csv(io.StringIO(r.text))
        if verbose:
            print(f"  Total time: {time.time() - start:0.3} sec")
    else:
        start = time.time()
        if verbose:
            print(f"Reading the file {local_filename} from disk")
        df = pd.read_csv(local_filename)
        if verbose:
            print(f"  Total time: {time.time() - start:0.3} sec")
    return df

In [27]:
df = get_and_parse_cached_remote_csvfile(external_file_location)

Reading the file galaxy1000.csv from disk
  Total time: 0.0227 sec


In [28]:
df = get_and_parse_cached_remote_csvfile(external_file_location, local_filename="test.csv", verbose=False)

Let's take a look at a similar workflow for external queries. 

In [29]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

In [30]:
query = """ -- Get some nearby sources
        SELECT top 1000 
            ra,dec,source_id,parallax,pm
        FROM gaiaedr3.gaia_source 
        WHERE parallax_over_error > 15.0
            AND parallax > 200.0
        ORDER BY parallax DESC"""

In [31]:
def get_gaia_query(q):
    start = time.time()
    job = Gaia.launch_job(q)
    print(f"Total time: {time.time()-start:0.2f} sec")
    return job.get_results()

result_table = get_gaia_query(query)
print(result_table)

Total time: 1.08 sec
        ra                 dec         ...      parallax          pm   
       deg                 deg         ...        mas          mas / yr
------------------ ------------------- ... ------------------ ---------
217.39232147200883  -62.67607511676666 ...  768.0665391873573  3859.228
269.44850252543836   4.739420051112412 ...   546.975939730948 10393.349
164.10319030755974   7.002726940984864 ... 415.17941567802137 4715.3296
165.83095967577933  35.948653032660104 ... 392.75294543876464   4811.68
101.28662552099249 -16.720932526023173 ... 374.48958852876103  1024.399
24.771674208211856 -17.947682860008488 ...  373.8443122683992 3231.9072
24.771554293454546 -17.948299887129313 ... 367.71189618147696 3428.8074
 282.4587890175222  -23.83709744872712 ...  336.0266016683708  668.1399
 355.4800152581559  44.170375700747755 ... 316.48118678226916 1595.6226
 53.22829341517546  -9.458168216292322 ...  310.5772928005821  974.9817
               ...                 ... ... 

Great: we can use `result_table` in the downstream analysis. The problem of course is that we rerun our notebook we have to download the data again (and the computation on the remote side needs to happen as well. This isn't ecofriendly!)

<font size=+1 color=red>**Question**</font>: what's the problem if we just use the same caching strategy in the CSV download example above?


We might be better off memorizing the result for a given input and save the data locally. We could do this "by hand" but [`joblib`](https://joblib.readthedocs.io/en/latest/) was built for just this (and parallel computing):

<pre>
    The Memory class defines a context for lazy evaluation of function, by putting the results in a store, by default using a disk, and not re-running the function twice for the same arguments. It works by explicitly saving the output to a file and it is designed to work with non-hashable and potentially large input and output data types such as numpy arrays.
</pre>

In [32]:
from joblib import Memory
cachedir = './joblib_cache'
memory = Memory(cachedir, verbose=0, bytes_limit=1e7)

In [33]:
@memory.cache
def get_gaia_query(q):
    start = time.time()
    job = Gaia.launch_job(q)
    print(f"Total time: {time.time()-start:0.2f} sec")
    return job.get_results()

Use `memory` as a function decorator. Every time `get_gaia_query` is called now, the variable `q` will be checked. If we've seen this before, then joblib will just return the output from last time. Otherwise we'll run this function and save the results in the `joblib_cache` directory.

In [34]:
result_table = get_gaia_query(query)

Total time: 1.05 sec


In [35]:
%time result_table = get_gaia_query(query)

CPU times: user 5.9 ms, sys: 2.69 ms, total: 8.59 ms
Wall time: 13.4 ms


<font size=+1 color=red>**Question**</font>: In what case's might you NOT want to cache query results?

In [36]:
def get_weather_alerts(area="CA"):
    r = requests.get(f"https://api.weather.gov/alerts/active?area={area}")
    zone = r.json()["features"][0]["properties"]["areaDesc"]
    return f'{zone}: {r.json()["features"][0]["properties"]["headline"]}'

In [37]:
print(get_weather_alerts())

Northern Trinity: Air Quality Alert issued August 16 at 10:05AM PDT by NWS Eureka CA
